## 1.环境配置

In [ ]:
# 准备工作
# 选择Scanpy内核 + vscode
import numpy as np
import pandas as pd
import scanpy as sc
import scrublet as scr
import matplotlib.pyplot as plt
import scvi
import warnings
import os

warnings.simplefilter(action='ignore', category=Warning) # 设置警告过滤器，忽略所有警告。这样做可以使输出结果更清晰，不被警告信息干扰
sc.settings.verbosity = 3 # 设置 Scanpy 的输出详细程度为 3（较高的详细程度）。这样设置可以让 Scanpy 在处理数据时提供更多的信息输出，帮助用户更好地了解数据处理过程
sc.settings.set_figure_params(dpi=80) # 设置 Scanpy 绘图的 DPI（每英寸点数）为 80

## 2.数据读取

In [ ]:
# 读取数据
# 因为我们表达矩阵观测是基因symbol，变量为细胞ID，所以我们需要转置，因为我们的adata对象要求观测是细胞ID，变量为基因symbol
adata = sc.read_csv('data/data01/raw_counts/GSM5226574_C51ctr_raw_counts.csv').T
adata

# 查看数据维度
adata.X.shape
# (6099, 34546)

# 导出原始表达文件，作为中间文件
adata.write('original_adata.h5ad')

## 3.去除双细胞

In [ ]:
# 加载h5ad文件
adata = sc.read('original_adata.h5ad')

# Doublet过滤
# 过滤低表达基因以及高变基因选择
sc.pp.filter_genes(adata, min_cells = 10)
sc.pp.highly_variable_genes(adata, n_top_genes = 2000, subset = True, flavor = 'seurat_v3')
adata.shape

# 训练模型
scvi.model.SCVI.setup_anndata(adata) # 设置AnnData对象，使其适合使用SCVI模型进行训练。
vae = scvi.model.SCVI(adata) # 实例化一个SCVI模型，并将其与预处理后的AnnData对象关联起来
vae.train() # 训练SCVI模型，使其能够捕捉和学习单细胞数据中的潜在结构和模式
# SCVI模型是一种深度生成模型，特别适合处理单细胞RNA测序数据中的噪声和复杂结构
# 使用训练好的SCVI模型初始化SOLO模型
solo = scvi.external.SOLO.from_scvi_model(vae) # scvi.external.SOLO 是SOLO工具的类，用于双重细胞检测
solo.train()

df = solo.predict() # # 使用SOLO模型进行预测
df['prediction'] = solo.predict(soft = False) # # 进行硬预测，即将预测结果转换为标签
df.index = df.index.map(lambda x: x[:-2]) # # 修改索引(df.index.map(lambda x: x[:-2]) 使用一个lambda函数将每个索引值的最后两个字符去掉。)
df

In [ ]:
# 查看一下对象的维度
adata.shape

# 查看一下有多少细胞被定义为双细胞
df.groupby('prediction').count()
#             doublet  singlet
# prediction                  
# doublet        1245     1245
# singlet        4854     4854

# 使用scvi进行双细胞检测的流程
## 1. 读取数据
## 2. 过滤低表达基因以及高变基因选择
## 3. 训练模型
## 4. 预测双细胞
## 5. 查看结果
## 因为scvi运行的时间比较长，所以我们需要保存中间文件，以便下次可以直接加载中间文件，不用重新运行scvi模型

In [ ]:
# 这里直接使用scvi的预测结果，会导致预测的双包细胞太多
# 所以这里我们计算每个细胞的双包细胞概率，然后根据概率选择双包细胞
df['dif'] = df.doublet - df.singlet
df
import seaborn as sns
import matplotlib.pyplot as plt

sns.displot(df[df.prediction == 'doublet'], x = 'dif') # 根据这个可视化选择阈值为1，其实有一点考虑就是，分布图上在1左右的区间呈现明显的下坠趋势，可能就是一个自然的阈值选择
plt.savefig(dir+"01-df-displot.png")

# 选择双包细胞
doublets = df[(df.prediction == 'doublet') & (df.dif > 1)]
doublets # 获取双细胞ID

# 得到双细胞ID后我们这里就可以重新读取表达矩阵并进行后续常规流程分析
adata = sc.read_csv('data/data01/raw_counts/GSM5226574_C51ctr_raw_counts.csv').T
adata
adata.obs['doublet'] = adata.obs.index.isin(doublets.index)
# 在 adata 的观测数据（细胞元数据）中新增一列 doublet。这个新列的值是布尔类型，指示每个细胞是否在 doublets 数据框的索引中
# adata.obs.index.isin(doublets.index) 方法用于检查 adata 中的每个细胞索引是否存在于 doublets 的索引中，并返回布尔值
adata.obs

adata = adata[~adata.obs.doublet] # 用按位取反运算符 ~ 将布尔值反转，即将 True 变为 False，False 变为 True
# 从 adata 对象中删除被标记为双细胞的细胞
adata
# View of AnnData object with n_obs × n_vars = 5618 × 34546
#    obs: 'doublet'
# 一开始用6099个细胞，现在剩下5618个细胞

## 4.单细胞标准分析流程

In [ ]:
# 然后接下来我们开始进行单细胞标准的分析流程
# 计算线粒体基因比例
adata.var['mt'] = adata.var.index.str.startswith('MT-')
adata.var

# 核糖体RNA基因
import pandas as pd
ribo_url = "http://software.broadinstitute.org/gsea/msigdb/download_geneset.jsp?geneSetName=KEGG_RIBOSOME&fileType=txt"
ribo_genes = pd.read_table(ribo_url, skiprows=2, header = None)
ribo_genes

adata.var['ribo'] = adata.var_names.isin(ribo_genes[0].values)
# 这段代码用于标记 adata 数据中哪些基因是核糖体RNA基因
# adata.var_names 是一个包含 adata 中所有基因名称的索引对象
# ribo_genes[0].values 提取了之前从 URL 读取的 ribo_genes 数据框中的核糖体RNA基因名称
# isin 方法用于检查 adata 中的基因名称是否存在于 ribo_genes 列表中，并返回一个布尔数组
# adata.var['ribo'] 在变量（基因）数据框 adata.var 中新增一列 ribo。这列的值是布尔类型，指示每个基因是否是核糖体RNA基因

In [ ]:
sc.pp.calculate_qc_metrics(adata, qc_vars=['mt', 'ribo'], percent_top=None, log1p=False, inplace=True)
# 计算质量控制（QC）指标，用于评估数据质量。
# adata 是包含单细胞数据的 AnnData 对象。
# qc_vars=['mt', 'ribo'] 指定要计算的QC变量，这里是线粒体基因（mt）和核糖体基因（ribo）。
# percent_top=None 指定不计算表达量最高的基因的百分比。
# log1p=False 指定不对数据进行对数转换。
# inplace=True 表示直接在 adata 对象中添加计算的QC指标

adata.var.sort_values('n_cells_by_counts')
# 按照 n_cells_by_counts 列对 adata.var 数据框进行排序。
# n_cells_by_counts 表示每个基因在多少个细胞中被检测到的计数。
# 这个排序可以帮助识别那些在大多数细胞中表达的基因，以及那些在少数细胞中表达的基因

# 过滤基因
sc.pp.filter_genes(adata, min_cells=3) # 过滤在少于指定数量的细胞中表达的基因。这里 min_cells=3 表示基因必须在至少 3 个细胞中表达，否则会被过滤掉
adata.var.sort_values('n_cells_by_counts')
adata.obs.sort_values('n_genes_by_counts')

# 绘制小提琴图
sc.pl.violin(adata, ['n_genes_by_counts', 'total_counts', 'pct_counts_mt', 'pct_counts_ribo'], jitter=0.4, multi_panel=True)
plt.savefig(dir+"02-qc_violin.png")

# 过滤细胞
import numpy as np
upper_lim = np.quantile(adata.obs.n_genes_by_counts.values, .98) # 计算 n_genes_by_counts 列的第 98 个百分位数（即上限）
#upper_lim = 3000
upper_lim # 相当于98%的细胞表达的最高的基因数量
adata = adata[adata.obs.n_genes_by_counts < upper_lim] # 过滤掉那些基因数目过高的细胞（当然也可以直接使用默认值3000）
adata.obs
adata = adata[adata.obs.pct_counts_mt < 20] # 高比例的线粒体基因表达通常与细胞压力或死亡相关，可能是低质量细胞的指示。
adata = adata[adata.obs.pct_counts_ribo < 2] # 高比例的核糖体RNA基因表达可能是技术噪音的结果。
adata 
# View of AnnData object with n_obs × n_vars = 5489 × 24080
#     obs: 'doublet', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'total_counts_ribo', 'pct_counts_ribo'
#     var: 'mt', 'ribo', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts', 'n_cells'

In [ ]:
# 过滤后的每个细胞的总计数
adata.X.sum(axis = 1) # 计算每个细胞的总计数（UMI count）
# adata.X 是包含基因表达数据的稀疏矩阵，axis=1 表示按行（即每个细胞）进行求和

#normalize every cell to 10,000 UMI
sc.pp.normalize_total(adata, target_sum=1e4) 
adata.X.sum(axis = 1)

#change to log counts
sc.pp.log1p(adata) 
adata.X.sum(axis = 1)

adata.raw = adata # 保存当前处理后的数据到 adata.raw 中，保留原始未处理的数据

In [ ]:
# 高变基因选择以及可视化
sc.pp.highly_variable_genes(adata, n_top_genes = 2000)
sc.pl.highly_variable_genes(adata)
plt.savefig(dir+"03-highly_variable_genes.png")

In [ ]:
# 选择主成分
adata = adata[:, adata.var.highly_variable] # 选择 adata 中的高变基因
sc.pp.regress_out(adata, ['total_counts', 'pct_counts_mt', 'pct_counts_ribo']) # 用于对数据进行回归校正，去除潜在的混杂因素
sc.pp.scale(adata, max_value=10) # 归一化
sc.tl.pca(adata, svd_solver='arpack')
sc.pl.pca_variance_ratio(adata, log=True, n_pcs = 50)
plt.savefig(dir+"04-pca_variance.png")

sc.pp.neighbors(adata, n_pcs = 30)
sc.tl.umap(adata)
sc.pl.umap(adata)
plt.savefig(dir+"05-umap.png")

In [ ]:
sc.tl.leiden(adata, resolution = 0.5)
sc.pl.umap(adata, color=['leiden'])
plt.savefig(dir+"05-umap_leiden.png")

# 至此，单样本分析流程结束
# 后续我们会根据这个单样本分析流程来进行多样本分析流程